In [21]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import re
import requests

options = webdriver.ChromeOptions()
# Run Chrome in headless mode
#options.add_argument('--headless')
options.add_argument("--disable-search-engine-choice-screen")
# Create the driver (using Chrome)
driver = webdriver.Chrome(executable_path="chromedriver-win64/chromedriver.exe",options=options)

# Access the site
driver.get("https://www.hs.fi/politiikka/")
# Wait for the page to load
driver.implicitly_wait(2)
# Find and swtivh to the correct iframe
iframe = driver.find_element(By.XPATH, '//iframe[@title="SP Consent Message"]')
driver.switch_to.frame(iframe)
# Find button to conent to cookies
button = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, '//button[@title="OK"]'))
)
# Click the said button
button.click()
# Scroll to the bottom to dynamically load everything
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# Wait for the page to fully load
sleep(5)

C:\Users\joona\AppData\Local\Temp\ipykernel_15132\608407366.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="chromedriver-win64/chromedriver.exe",options=options)


In [20]:
# Get HTML
page_source = driver.page_source
# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, "html.parser")
# Extract data using BeautifulSoup methods
data_element = soup.find_all("a",attrs={"class":"block"})
print(len(data_element), "articles found")
# Quit the driver
driver.quit()

110 articles found


In [23]:
a_list = []
for article in data_element:
    # Use regular expressions to extract the article numbers
    item = re.findall(r"([0-9]+).html",str(article))[0]
    a_list.append(item)

In [64]:
# Save article numbers as a text file
with open("article_numbers.txt","w") as file:
    for item in a_list:
        file.write(item+"\n")

In [77]:
json_list = []
for article in a_list:
    # Use the API to get comments from each of the article
    url = f"https://www.hs.fi/api/commenting/hs/articles/{article}/comments?sort=newest&onlyCommentsWithRealNames=false"
    comments = requests.get(url)
    json_list.append(comments.json())

In [83]:
# Save the comments as JSON
with open("articles.json","w") as file:
    for item in json_list:
        file.write(str(item)+"\n")